In [1]:
import numpy as np 
import pandas as pd 
import pyspark.sql.types as T
import pyspark.sql.functions as F
from pyspark.sql import SparkSession
from pyspark.sql.window import Window

In [2]:
# initialize spark session
spark = SparkSession.builder \
            .master("local[*]") \
            .appName("ShortNSimple") \
            .getOrCreate()
spark

Data Source: https://www.kaggle.com/eidanch/counties-geographic-coordinates

In [3]:
pd.read_csv("datasets/countries.csv").head()

,country,latitude,longitude,location
0,AD,42.546245,1.601554,Andorra
1,AE,23.424076,53.847818,United Arab Emirates
2,AF,33.939110,67.709953,Afghanistan
3,AG,17.060816,-61.796428,Antigua and Barbuda
4,AI,18.220554,-63.068615,Anguilla


In [4]:
data = spark.createDataFrame(pd.read_csv("datasets/countries.csv").dropna())
data.show(2)

+-------+---------+------------------+--------------------+
|country| latitude|         longitude|            location|
+-------+---------+------------------+--------------------+
|     AD|42.546245|1.6015540000000001|             Andorra|
|     AE|23.424076|53.847818000000004|United Arab Emirates|
+-------+---------+------------------+--------------------+
only showing top 2 rows



In [5]:
countries_name = data.selectExpr("country as county_code", "location as county_name").limit(10)
countries_name.show(2)

+-----------+--------------------+
|county_code|         county_name|
+-----------+--------------------+
|         AD|             Andorra|
|         AE|United Arab Emirates|
+-----------+--------------------+
only showing top 2 rows



In [6]:
countries_lot_lon = data.selectExpr("country as county_code", "latitude", "longitude").limit(15)
countries_lot_lon = countries_lot_lon.sample(fraction=0.66, withReplacement=False)
countries_lot_lon.show(2)

+-----------+---------+------------------+
|county_code| latitude|         longitude|
+-----------+---------+------------------+
|         AD|42.546245|1.6015540000000001|
|         AE|23.424076|53.847818000000004|
+-----------+---------+------------------+
only showing top 2 rows



In [7]:
countries_lot_lon.count()

11

## Apply Join

In [8]:
merged_dataset = countries_name.join(
    countries_lot_lon,
    on=countries_name.county_code == countries_lot_lon.county_code,
    how="full"
)
merged_dataset.show(15, False)

+-----------+--------------------+-----------+-------------------+-------------------+
|county_code|county_name         |county_code|latitude           |longitude          |
+-----------+--------------------+-----------+-------------------+-------------------+
|AD         |Andorra             |AD         |42.546245          |1.6015540000000001 |
|AE         |United Arab Emirates|AE         |23.424076          |53.847818000000004 |
|AF         |Afghanistan         |null       |null               |null               |
|AG         |Antigua and Barbuda |AG         |17.060816          |-61.796428000000006|
|AI         |Anguilla            |AI         |18.220554          |-63.068615         |
|AL         |Albania             |AL         |41.153332          |20.168331          |
|AM         |Armenia             |null       |null               |null               |
|AN         |Netherlands Antilles|null       |null               |null               |
|AO         |Angola              |AO       

## Apply Coalesce

In [9]:
merged_dataset = merged_dataset.withColumn(
    "new_county_code",
    F.coalesce(countries_name.county_code, countries_lot_lon.county_code, F.lit("Default"))
)

merged_dataset.show(15, False)

+-----------+--------------------+-----------+-------------------+-------------------+---------------+
|county_code|county_name         |county_code|latitude           |longitude          |new_county_code|
+-----------+--------------------+-----------+-------------------+-------------------+---------------+
|AD         |Andorra             |AD         |42.546245          |1.6015540000000001 |AD             |
|AE         |United Arab Emirates|AE         |23.424076          |53.847818000000004 |AE             |
|AF         |Afghanistan         |null       |null               |null               |AF             |
|AG         |Antigua and Barbuda |AG         |17.060816          |-61.796428000000006|AG             |
|AI         |Anguilla            |AI         |18.220554          |-63.068615         |AI             |
|AL         |Albania             |AL         |41.153332          |20.168331          |AL             |
|AM         |Armenia             |null       |null               |null   

### SQL

In [10]:
countries_name.registerTempTable("countries_name")
countries_lot_lon.registerTempTable("countries_lot_lon")

In [11]:
sql_merged_data = spark.sql(
    """
        SELECT
            a.county_code as a_county_code, 
            b.county_code as b_county_code,
            coalesce(a.county_code, b.county_code, "DEFAULT") as new_county_code
        FROM
            countries_name a
        FULL JOIN
            countries_lot_lon b
        ON
            a.county_code = b.county_code
    """
)

sql_merged_data.show(15, False)

+-------------+-------------+---------------+
|a_county_code|b_county_code|new_county_code|
+-------------+-------------+---------------+
|AD           |AD           |AD             |
|AE           |AE           |AE             |
|AF           |null         |AF             |
|AG           |AG           |AG             |
|AI           |AI           |AI             |
|AL           |AL           |AL             |
|AM           |null         |AM             |
|AN           |null         |AN             |
|AO           |AO           |AO             |
|AQ           |AQ           |AQ             |
|null         |AR           |AR             |
|null         |AS           |AS             |
|null         |AT           |AT             |
|null         |AU           |AU             |
+-------------+-------------+---------------+

